# *Fingerprints* moleculares e similaridade

*Fingerprints* moleculares são formas de representação molecular. São vetores numéricos cujos valores indicam aspectos estruturais de uma molécula, como a presença de certos átomos, grupos funcionais e subestruturas. Para uma introdução ao cálculo de *fingerprints* no RDKit veja o Notebook [*Fingerprints* moleculares no RDKit](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Fingerprints%20moleculares%20no%20RDKit.ipynb)

Pela forma como os *fingerprints* são construídos, compostos similares tendem a apresentar muitas subestruturas em comum e, consequentemente, seus *fingerprints* tendem a ser similares. Ainda que o termo "similar" possa ser usado de forma vaga no dia a dia, na quiminformática, a **similaridade molecular** envolve medidas **quantitativas** da semelhança entre duas estruturas químicas.

Existem várias formas de medir a similaridade entre *fingerprints* moleculares, cada uma com suas vantagens e desvantagens. Alguns exemplos de métricas mais usadas são descritas a seguir:

1. **Índice de Tanimoto** (também chamado coeficiente de Tanimoto, coeficiente de similaridade de Jaccard, índice de Jaccard): uma das métricas mais usadas, sendo específica para *fingerprints* binários. Mede a razão entre a interseção e a união de dois conjuntos, variando de 0 (nenhuma similaridade, todos os *bits* diferentes) a 1 (máxima similaridade, todos os *bits* iguais). Podemos dizer que somente os verdadeiros positivos (*bits* iguais a 1 em ambos os *fingerprints*) impactam o índice de Tanimoto.
   
2. **Similaridade de Dice** (coeficiente de Dice-Sørensen): calcula-se como duas vezes o número de valores positivos em comum nos conjuntos dividido pela soma dos tamanhos dos conjuntos. É equivalente ao *F1-score* usado no aprendizado de máquina, dessa forma, também leva em conta os *bits* que diferem entre os *fingerprints* para determinar sua similaridade.

3. **Similaridade Cosseno** (cosine similarity): relacionada ao ângulo entre os vetores correspondentes aos *fingerprints*. Dessa forma, não depende da magnitude e pode ser usada para *fingerprints* contínuos ou binários, sendo útil para a comparação de *fingerprints* de tipos diferentes. Seu valor varia de -1 (vetores opostos) a 1 (vetores na mesma direção).

4. **Distância Euclidiana**: mede a distância linear entre dois vetores no espaço multidimensional, sendo mais adequada para *fingerprints* contínuos. É sensível à magnitude dos valores dos *fingerprints*. Além disso, seu uso deve ser evitado para *fingerprints* binários, pois tanto a presença quanto a ausência de subestruturas em comum contribui para diminuir a distância Euclidiana.

5. **Índice de Tversky**: uma generalização dos índices de Tanimoto e Dice, inclui dois termos `a` e `b` que correspondem a "pesos" que permitem dar mais importância a subestruturas presentes em um dos *fingerprints*. Quando `a` = `b` = 1, temos o índice de Tanimoto, e quando `a` = `b` = 0.5, temos a similaridade de Dice. Em geral, usamos valores tais que `a` + `b` = 1. 

No RDKit, diversas métricas estão disponíveis através do módulo `DataStructs`: Tanimoto, Dice, Cosine, Sokal, Russel, Kulczynski, McConnaughey e Tversky. Em geral, o índice de Tanimoto é o mais utilizado.

A medida da similaridade entre *fingerprints* moleculares encontra diversas aplicações:

- Rápida identificação de estruturas químicas em bancos de dados e realização de triagens virtuais para busca de compostos similares a um composto de interesse
- Desenvolvimento de modelos de aprendizado de máquina (compostos semelhantes tendem a apresentar propriedades semelhantes)
- Agrupamento de estruturas químicas (*clustering*)

Vamos mostrar neste Notebook como calcular a similaridade entre diferentes tipos de *fingerprints* moleculares usando o RDKit.

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator

In [2]:
# Vamos usar um pequeno banco de estruturas:
# atorvastatina, sinvastatina, aspirina
ms = [Chem.MolFromSmiles('CC(C)C1=C(C(=C(N1CC[C@H](C[C@H](CC(=O)O)O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4'), 
      Chem.MolFromSmiles('CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C@H](C=C2)C)CC[C@@H]3C[C@H](CC(=O)O3)O)C'), 
      Chem.MolFromSmiles('CC(=O)OC1=CC=CC=C1C(=O)O')]

Vamos agora gerar os *fingerprints* usando um objeto `FingeprintGenerator`. Note que usamos o método `GetFingerprint()` para gerar o *fingerprint* no formato apropriado para uso na função de similaridade.

In [3]:
# Criando o objeto FingerprintGenerator. Vamos usar o fingerprint RDKit
rdkgen = rdFingerprintGenerator.GetRDKitFPGenerator(fpSize=1024)

In [4]:
# Gerar fingerprints
fps = [rdkgen.GetFingerprint(x) for x in ms]

In [5]:
# Veja o tipo do objeto fingerprint
type(fps[0])

rdkit.DataStructs.cDataStructs.ExplicitBitVect

## Funções "Similarity" do RDKit

Escrevi a função abaixo para condensar todas as métricas de similaridade disponíveis no RDKit

In [6]:
from rdkit import DataStructs

def calc_similarity(fp1, fp2, metric="Tanimoto", a=0.5, b=0.5):
    """
    Calcula a similaridade entre dois fingerprints moleculares usando diferentes métricas de similaridade 
    disponíveis no RDKit.

    Parâmetros:
    -----------
    fp1 : objeto do tipo ExplicitBitVect ou SparseBitVect do RDKit
        O primeiro fingerprint molecular para comparação.
        
    fp2 : objeto do tipo ExplicitBitVect ou SparseBitVect do RDKit
        O segundo fingerprint molecular para ser comparado a fp1.
        
    metric : str, padrão "Tanimoto"
        A métrica de similaridade a ser utilizada para a comparação. Opções disponíveis:
        - "AllBit": Similaridade com base em todos os bits da impressão digital.
        - "Asymmetric": Similaridade assimétrica.
        - "BraunBlanquet": Índice de Braun-Blanquet.
        - "Cosine": Similaridade cosseno.
        - "Dice": Similaridade Dice.
        - "Kulczynski": Índice de Kulczynski.
        - "McConnaughey": Similaridade de McConnaughey.
        - "OnBit": Similaridade com base nos bits "ligados" (iguais a 1).
        - "RogotGoldberg": Similaridade de Rogot-Goldberg.
        - "Russel": Similaridade Russel.
        - "Sokal": Similaridade Sokal.
        - "Tanimoto": Índice de Tanimoto (Jaccard), o mais usado para cálculo de similaridade de fingerprints.
        - "Tversky": Índice de Tversky, permite uma ponderação assimétrica da similaridade.

    a : float, padrão 0.5
        O parâmetro alpha utilizado no cálculo da similaridade de Tversky. Ajusta a importância
        das características únicas de fp1. Usado apenas se metric="Tversky".

    b : float, padrão 0.5
        O parâmetro beta utilizado no cálculo da similaridade de Tversky. Ajusta a importância
        das características únicas de fp2. Usado apenas se metric="Tversky".
    
    Retorna:
    --------
    float
        O valor de similaridade entre os dois fingerprints moleculares, baseado na métrica selecionada.
    """
    
    if metric == "AllBit":
        return DataStructs.AllBitSimilarity(fp1, fp2)
    elif metric == "Asymmetric":
        return DataStructs.AsymmetricSimilarity(fp1, fp2)
    elif metric == "BraunBlanquet":
        return DataStructs.BraunBlanquetSimilarity(fp1, fp2)
    elif metric == "Cosine":
        return DataStructs.CosineSimilarity(fp1, fp2)
    elif metric == "Dice":
        return DataStructs.DiceSimilarity(fp1, fp2)
    elif metric == "Kulczynski":
        return DataStructs.KulczynskiSimilarity(fp1, fp2)
    elif metric == "McConnaughey":
        return DataStructs.McConnaugheySimilarity(fp1, fp2)
    elif metric == "OnBit":
        return DataStructs.OnBitSimilarity(fp1, fp2)
    elif metric == "RogotGoldberg":
        return DataStructs.RogotGoldbergSimilarity(fp1, fp2)
    elif metric == "Russel":
        return DataStructs.RusselSimilarity(fp1, fp2)
    elif metric == "Sokal":
        return DataStructs.SokalSimilarity(fp1, fp2)
    elif metric == "Tanimoto":
        return DataStructs.TanimotoSimilarity(fp1, fp2) 
    elif metric == "Tversky":
        # O índice de Tversky requer o uso de "pesos" a e b
        # Por exemplo, ao comparar dois *fingerprints* `fp1` e `fp2`, damos mais importância
        # às subestruturas presentes em `fp1` usando TverskySimilarity(fp1, fp2, a=0.7, b=0.3)
        return DataStructs.TverskySimilarity(fp1, fp2, a=a, b=b)
    else:
        raise ValueError(f"Escolha uma métrica válida: [AllBit, Asymmetric, BraunBlanquet, Cosine, Dice, Kulczynski, McConnaughey, OnBit, RogotGoldberg, Russel, Sokal, Tanimoto, Tversky]")


In [7]:
similarity_metrics = ["AllBit", "Asymmetric", "BraunBlanquet", "Cosine", "Dice", 
                      "Kulczynski", "McConnaughey", "OnBit", "RogotGoldberg", 
                      "Russel", "Sokal", "Tanimoto", "Tversky"]

In [8]:
for m in similarity_metrics:
    print(f"{m} similarity: {calc_similarity(fps[0], fps[1], metric=m):.3f}")

AllBit similarity: 0.513
Asymmetric similarity: 0.931
BraunBlanquet similarity: 0.510
Cosine similarity: 0.689
Dice similarity: 0.659
Kulczynski similarity: 0.720
McConnaughey similarity: 0.441
OnBit similarity: 0.491
RogotGoldberg similarity: 0.403
Russel similarity: 0.471
Sokal similarity: 0.326
Tanimoto similarity: 0.491
Tversky similarity: 0.659


In [9]:
for m in similarity_metrics:
    print(f"{m} similarity: {calc_similarity(fps[0], fps[2], metric=m):.3f}")

AllBit similarity: 0.348
Asymmetric similarity: 0.934
BraunBlanquet similarity: 0.315
Cosine similarity: 0.543
Dice similarity: 0.472
Kulczynski similarity: 0.625
McConnaughey similarity: 0.250
OnBit similarity: 0.308
RogotGoldberg similarity: 0.310
Russel similarity: 0.291
Sokal similarity: 0.182
Tanimoto similarity: 0.308
Tversky similarity: 0.472


## Matriz de similaridades/distâncias

Podemos usar a função `calc_similarity()` para calcular as similaridades entre todos os pares de *fingerprints* em um conjunto de dados. Com isso, podemos criar uma **matriz de similaridades** ou uma **matriz de distâncias** (distância = 1 - similaridade).

A matriz de distâncias é útil para aplicações como *clustering* ou determinação do domínio de aplicabilidade. Veja um exemplo no Notebook [Clustering (Agrupamento)](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Clustering%20(Agrupamento).ipynb).

In [10]:
# Cálculo da matriz de distâncias 

# Iniciamos uma matriz n x n preenchida de zeros, n é o número total de estruturas
n = len(fps)
dist_mat = np.zeros((len(fps), len(fps)))

# Como a matriz é simétrica, podemos calcular apenas metade dela:
for i in range(n):
    for j in range(i):
        sim = DataStructs.TanimotoSimilarity(fps[i], fps[j])
        # distância = 1 - similaridade
        dist = 1 - sim
        dist_mat[i][j] = dist
        
# E preenchemos o restante usando a matriz transposta
# Note que a diagonal permanece preenchida por zeros
# já que a distância entre um ponto e ele próprio é zero
dist_mat = dist_mat + dist_mat.T

In [11]:
dist_mat

array([[0.        , 0.50866463, 0.69151139],
       [0.50866463, 0.        , 0.76730486],
       [0.69151139, 0.76730486, 0.        ]])

## Funções "Bulk" do RDKit

As funções para cálculo de similaridade também estão disponíveis na versão "Bulk", que são formas otimizadas para calcular as similaridades entre um *fingerprint* e uma lista de *fingerprints*.

In [12]:
from rdkit import DataStructs

def calc_bulk_similarity(fp1, fps, metric="Tanimoto", a=0.5, b=0.5):
    """
    Calcula a similaridade "Bulk" entre um fingerprint (fp1) e uma lista de fingerprints (fps).
    
    Parâmetros:
        fp1: O fingerprint de referência.
        fps: Lista de fingerprints para comparar com fp1.
        metric: A métrica de similaridade a ser utilizada para a comparação (padrão = "Tanimoto").
        a: alpha (usado apenas quando metric="Tversky").
        b: beta (usado apenas quando metric="Tversky").
    
    Retorna:
        Uma lista de similaridades de mesmo tamanho da lista de fingerprints.
    """
    
    if metric == "AllBit":
        sim = DataStructs.BulkAllBitSimilarity(fp1, fps)
    elif metric == "Asymmetric":
        sim = DataStructs.BulkAsymmetricSimilarity(fp1, fps)
    elif metric == "BraunBlanquet":
        sim = DataStructs.BulkBraunBlanquetSimilarity(fp1, fps)
    elif metric == "Cosine":
        sim = DataStructs.BulkCosineSimilarity(fp1, fps)
    elif metric == "Dice":
        sim = DataStructs.BulkDiceSimilarity(fp1, fps)
    elif metric == "Kulczynski":
        sim = DataStructs.BulkKulczynskiSimilarity(fp1, fps)
    elif metric == "McConnaughey":
        sim = DataStructs.BulkMcConnaugheySimilarity(fp1, fps)
    elif metric == "OnBit":
        sim = DataStructs.BulkOnBitSimilarity(fp1, fps)
    elif metric == "RogotGoldberg":
        sim = DataStructs.BulkRogotGoldbergSimilarity(fp1, fps)
    elif metric == "Russel":
        sim = DataStructs.BulkRusselSimilarity(fp1, fps)
    elif metric == "Sokal":
        sim = DataStructs.BulkSokalSimilarity(fp1, fps)
    elif metric == "Tanimoto":
        sim = DataStructs.BulkTanimotoSimilarity(fp1, fps)
    elif metric == "Tversky":
        sim = DataStructs.BulkTverskySimilarity(fp1, fps, a=a, b=b)
    else:
        raise ValueError(f"Escolha uma métrica válida: [AllBit, Asymmetric, BraunBlanquet, Cosine, Dice, Kulczynski, McConnaughey, OnBit, RogotGoldberg, Russel, Sokal, Tanimoto, Tversky]")

    return sim


In [13]:
# Calcular a similaridade cosseno entre a atorvastatina e todos os compostos da lista
calc_bulk_similarity(fps[0], fps, metric="Cosine")

[1.0, 0.688915972715367, 0.5427564813365878]

## Busca por similaridade em um banco de estruturas

Um exemplo de aplicação da função `calc_bulk_similarity()`: vamos procurar antibióticos similares à penicilina (Penicillin G). Para isso, usaremos a lista de antibióticos que disponibilizei na pasta `datasets`. Defini similaridade como um índice de Tanimoto maior que 0.8.

In [14]:
antibiotics = pd.read_csv("../datasets/antibiotics_SMILES.csv")

In [15]:
antibiotics_fps = [rdkgen.GetFingerprint(Chem.MolFromSmiles(smi)) for smi in antibiotics["SMILES"]]

In [16]:
# Definir query
query_smiles = 'CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'
query_fp = rdkgen.GetFingerprint(Chem.MolFromSmiles(query_smiles))

# Calcular similaridades pelo índice de Tanimoto
scores = DataStructs.BulkTanimotoSimilarity(query_fp, antibiotics_fps)

# Selecionar as entradas com similaridade acima de um valor de corte
threshold = 0.8

for i, score in enumerate(scores):
    if score >= threshold:
        print(f"{antibiotics.iloc[i]['name']}: similaridade com a penicilina G = {score:.3f}")

Cefadroxil: similaridade com a penicilina G = 0.801
Cefalexin: similaridade com a penicilina G = 0.800
Amoxicillin: similaridade com a penicilina G = 0.955
Ampicillin: similaridade com a penicilina G = 0.968
Azlocillin: similaridade com a penicilina G = 0.898
Carbenicillin: similaridade com a penicilina G = 0.938
Cloxacillin: similaridade com a penicilina G = 0.813
Dicloxacillin: similaridade com a penicilina G = 0.812
Flucloxacillin: similaridade com a penicilina G = 0.800
Methicillin: similaridade com a penicilina G = 0.866
Mezlocillin: similaridade com a penicilina G = 0.849
Nafcillin: similaridade com a penicilina G = 0.852
Oxacillin: similaridade com a penicilina G = 0.829
Penicillin G: similaridade com a penicilina G = 1.000
Penicillin V: similaridade com a penicilina G = 0.928
Piperacillin: similaridade com a penicilina G = 0.892
Temocillin: similaridade com a penicilina G = 0.824
Ticarcillin: similaridade com a penicilina G = 0.895


# Algumas fontes consultadas

[Thread: [Rdkit-discuss] BulkTanimotoSimilarity](https://sourceforge.net/p/rdkit/mailman/rdkit-discuss/thread/663770d4-b809-c599-e379-31f57380a1d0@gmail.com/)

[Chem LibreTexts: 6.1: Molecular Descriptors](https://chem.libretexts.org/Courses/Intercollegiate_Courses/Cheminformatics/06%3A_Molecular_Similarity/6.01%3A_Molecular_Descriptors)